**Run DeViSE on NELL with RDFS**
---
You can run other settings by changing the parameters of "dataset", "semantic type", "hidden_dims" and etc.

For example, run on NELL with RDFS+literal by changing "semantic type" into "rdfs_text" and hidden_dims" into 400.

More parameter details are attached in the end.

**1. Bind your Google Drive**

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**2. Import Package**

In [9]:
import random
import argparse
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from collections import defaultdict
from collections import deque
from torch.autograd import Variable
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import json
import os
import json
import numpy as np

**3. Parameters Setting**

In [10]:
parser = argparse.ArgumentParser()

parser.add_argument('--data_dir', default='/content/drive/MyDrive/ISWC_demo/ZS_KGC/data/')
parser.add_argument('--dataset', default='NELL')

parser.add_argument("--embed_model", default='TransE', type=str)
parser.add_argument("--max_neighbor", default=50, type=int, help='neighbor number of each entity')
parser.add_argument("--embed_dim", default=100, type=int, help='dimension of triple embedding')
parser.add_argument("--ep_dim", default=200, type=int, help='dimension of entity pair embedding')
parser.add_argument("--batch_size", default=64, type=int)
parser.add_argument("--batch_rela_num", default=4, type=int)
parser.add_argument("--train_times", default=7000, type=int)

parser.add_argument("--loss_every", default=50, type=int)
parser.add_argument("--eval_every", default=500, type=int)

parser.add_argument('--p', default=0.5, help='dropout', type=float)
parser.add_argument('--lr', default=1e-3, help='learning rate', type=float)
parser.add_argument('--wds', default=1e-5, help='', type=float)
parser.add_argument('--manual_seed', default=12345, help='', type=int)
parser.add_argument("--semantic_type", default='rdfs', help='the type of relation embedding to input, options: {text, rdfs, rdfs_hie, rdfs_cons, rdfs_text}')
parser.add_argument('--hidden_dims', type=int, default=300, help='size of the hidden units in DeViSE')

parser.add_argument('--gpu', default=0, help='gpu id', type=int)

args = parser.parse_known_args()[0]

args.data_path = os.path.join(args.data_dir, args.dataset)
args.output_dims = args.ep_dim

if args.manual_seed is None:
    args.manual_seed = random.randint(1, 10000)
print("Random Seed: ", args.manual_seed)

np.random.seed(args.manual_seed)
random.seed(args.manual_seed)
torch.manual_seed(args.manual_seed)
if torch.cuda.is_available():
    torch.cuda.set_device(args.gpu)
    print('using gpu {}'.format(args.gpu))
    torch.cuda.manual_seed_all(args.manual_seed)
    torch.backends.cudnn.deterministic = True
else:
    print("GPU is not available!")

Random Seed:  12345
using gpu 0


**4. Loading Data**

In [11]:
def load_semantic_embed(data_path, dataset, type):
    """
    Load Semantic Embeddings.

    Parameters
    ----------
    file_name : str
        Name of the semantic embedding file.
    type: str
        Type of semantic embeddings, including

    Returns
    -------
    embeddings : NumPy arrays
       the size is
    Examples
    --------
    """

    file_name = ''
    file_path = os.path.join(data_path, 'semantic_embeddings')
    if dataset == 'NELL':
        if type == 'rdfs':
            file_name = os.path.join(file_path, 'rela_matrix_rdfs_55000.npz')
        elif type == 'rdfs_hie':
            file_name = os.path.join(file_path, 'rela_matrix_rdfs_hie_60000.npz')
        elif type == 'rdfs_cons':
            file_name = os.path.join(file_path, 'rela_matrix_rdfs_cons_60000.npz')
        elif type == 'text':
            file_name = os.path.join(file_path, 'rela_matrix_text.npz')
        elif type == 'rdfs_text':
            file_name = os.path.join(file_path, 'rela_matrix_rdfs_55000_text140.npz')
        else:
            print("WARNING: invalid semantic embeddings type")
    elif dataset == 'Wiki':
        if type == 'rdfs':
            file_name = os.path.join(file_path, 'rela_matrix_rdfs_65000.npz')
        elif type == 'rdfs_hie':
            file_name = os.path.join(file_path, 'rela_matrix_rdfs_hie_60000.npz')
        elif type == 'rdfs_cons':
            file_name = os.path.join(file_path, 'rela_matrix_rdfs_cons_60000.npz')
        elif type == 'text':
            file_name = os.path.join(file_path, 'rela_matrix_text.npz')
        elif type == 'rdfs_text':
            file_name = os.path.join(file_path, 'rela_matrix_rdfs_65000_text140.npz')
        else:
            print("WARNING: invalid semantic embeddings type")





    if file_name:
        rela_embeddings = np.load(file_name)['relaM'].astype('float32')


    else:
        print('WARNING: invalid semantic embeddings file path')
    return rela_embeddings

def random_pick(some_list, probabilities):
    x = random.uniform(0,1)
    cumulative_probability = 0.0
    for item, item_probability in zip(some_list, probabilities):
        cumulative_probability += item_probability
        if x < cumulative_probability:break
    return item

def load_train_data(args, train_tasks, symbol2id, ent2id, e1rel_e2, rel2id, rela2label, rela_matrix):
    print('##LOADING CANDIDATES')
    rel2candidates = json.load(open(os.path.join(args.data_path, 'rel2candidates_all.json')))
    task_pool = sorted(train_tasks.keys())  # ensure the readout is the same

    # print(task_pool)

    while True:
        rel_batch, rel_neg_batch, query_pairs, query_left, query_right, false_pairs, false_left, false_right, labels = [], [], [], [], [], [], [], [], []
        random.shuffle(task_pool)
        if len(rel2candidates[task_pool[0]]) <= 20:
            continue
        if len(rel2candidates[task_pool[1]]) <= 20:
            continue
        for query in task_pool[:args.batch_rela_num]:
            # print(query)
            relation_id = rel2id[query]
            candidates = rel2candidates[query]

            if args.dataset == 'Wiki':
                if len(candidates) <= 20:
                    # print 'not enough candidates'
                    continue

            train_and_test = train_tasks[query]

            random.shuffle(train_and_test)

            all_test_triples = train_and_test

            if len(all_test_triples) == 0:
                continue


            if len(all_test_triples) < args.batch_size:
                query_triples = [random.choice(all_test_triples) for _ in range(args.batch_size)]
            else:
                query_triples = random.sample(all_test_triples, args.batch_size)

            query_pairs += [[symbol2id[triple[0]], symbol2id[triple[2]]] for triple in query_triples]

            query_left += [ent2id[triple[0]] for triple in query_triples]
            query_right += [ent2id[triple[2]] for triple in query_triples]

            label = rela2label[query]

            # generate negative samples
            false_pairs_ = []
            false_left_ = []
            false_right_ = []
            for triple in query_triples:
                e_h = triple[0]
                rel = triple[1]
                e_t = triple[2]
                while True:
                    noise = random.choice(candidates)
                    if noise in ent2id.keys(): # ent2id.has_key(noise):
                        if (noise not in e1rel_e2[e_h+rel]) and noise != e_t:
                            break
                false_pairs_.append([symbol2id[e_h], symbol2id[noise]])
                false_left_.append(ent2id[e_h])
                false_right_.append(ent2id[noise])

            false_pairs += false_pairs_
            false_left += false_left_
            false_right += false_right_

            rel_batch += [rel2id[query] for _ in range(args.batch_size)]

            neg_rel_batch = list()
            for _ in range(args.batch_size):
                while True:
                    neg_query = random.choice(task_pool)
                    if neg_query != query:
                        break

                neg_rel_batch.append(rel2id[neg_query])

            rel_neg_batch += neg_rel_batch


            labels += [rela2label[query]] * args.batch_size

        yield rela_matrix[rel_batch], rela_matrix[rel_neg_batch], query_pairs, query_left, query_right, false_pairs, false_left, false_right, labels


**5. Models (DeViSE and Feature Encoder)**

In [12]:
def weights_init(m):
    classname = m.__class__.__name__
    if 'Linear' in classname:
        init.xavier_normal_(m.weight.data)
        init.constant_(m.bias, 0.0)


class LayerNormalization(nn.Module):
    ''' Layer normalization module '''

    def __init__(self, d_hid, eps=1e-3):
        super(LayerNormalization, self).__init__()

        self.eps = eps
        self.a_2 = nn.Parameter(torch.ones(d_hid), requires_grad=True)
        self.b_2 = nn.Parameter(torch.zeros(d_hid), requires_grad=True)

    def forward(self, z):
        if z.size(1) == 1:
            return z

        mu = torch.mean(z, keepdim=True, dim=-1)
        sigma = torch.std(z, keepdim=True, dim=-1)
        ln_out = (z - mu.expand_as(z)) / (sigma.expand_as(z) + self.eps)
        ln_out = ln_out * self.a_2.expand_as(ln_out) + self.b_2.expand_as(ln_out)

        return ln_out

class SupportEncoder(nn.Module):
    """docstring for SupportEncoder"""
    def __init__(self, d_model, d_inner, dropout=0.1):
        super(SupportEncoder, self).__init__()
        self.proj1 = nn.Linear(d_model, d_inner)
        self.proj2 = nn.Linear(d_inner, d_model)
        self.layer_norm = LayerNormalization(d_model)

        init.xavier_normal_(self.proj1.weight)
        init.xavier_normal_(self.proj2.weight)

        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        output = self.relu(self.proj1(x))
        output = self.dropout(self.proj2(output))


        return self.layer_norm(output + residual)

class Extractor(nn.Module):
    """
    Matching metric based on KB Embeddings
    """

    def __init__(self, embed_dim, num_symbols, embed=None):
        super(Extractor, self).__init__()
        self.embed_dim = int(embed_dim)
        self.pad_idx = num_symbols
        self.symbol_emb = nn.Embedding(num_symbols + 1, embed_dim, padding_idx=num_symbols)
        self.num_symbols = num_symbols

        self.gcn_w = nn.Linear(self.embed_dim, int(self.embed_dim / 2))
        self.gcn_b = nn.Parameter(torch.FloatTensor(self.embed_dim))

        self.fc1 = nn.Linear(self.embed_dim, int(self.embed_dim / 2))
        self.fc2 = nn.Linear(self.embed_dim, int(self.embed_dim / 2))

        self.dropout = nn.Dropout(0.2)
        self.dropout_e = nn.Dropout(0.2)

        self.symbol_emb.weight.data.copy_(torch.from_numpy(embed))

        self.symbol_emb.weight.requires_grad = False

        d_model = self.embed_dim * 2
        self.support_encoder = SupportEncoder(d_model, 2 * d_model, dropout=0.2)
        # self.query_encoder = QueryEncoder(d_model, process_steps)

    def neighbor_encoder(self, connections, num_neighbors):
        '''
        connections: (batch, 200, 2)
        num_neighbors: (batch,)
        '''
        num_neighbors = num_neighbors.unsqueeze(1)
        entities = connections[:, :, 1].squeeze(-1)
        ent_embeds = self.dropout(self.symbol_emb(entities))  # (batch, 50, embed_dim)
        concat_embeds = ent_embeds

        out = self.gcn_w(concat_embeds)
        out = torch.sum(out, dim=1)  # (batch, embed_dim)
        out = out / num_neighbors
        return out.tanh()

    def entity_encoder(self, entity1, entity2):
        entity1 = self.dropout_e(entity1)
        entity2 = self.dropout_e(entity2)
        entity1 = self.fc1(entity1)
        entity2 = self.fc2(entity2)
        entity = torch.cat((entity1, entity2), dim=-1)
        return entity.tanh()  # (batch, embed_dim)

    def forward(self, query, support, query_meta=None, support_meta=None):
        '''
        query: (batch_size, 2)
        support: (few, 2)
        return: (batch_size, )
        '''
        query_left_connections, query_left_degrees, query_right_connections, query_right_degrees = query_meta
        support_left_connections, support_left_degrees, support_right_connections, support_right_degrees = support_meta

        query_e1 = self.symbol_emb(query[:, 0])  # (batch, embed_dim)
        query_e2 = self.symbol_emb(query[:, 1])  # (batch, embed_dim)
        query_e = self.entity_encoder(query_e1, query_e2)

        support_e1 = self.symbol_emb(support[:, 0])  # (batch, embed_dim)
        support_e2 = self.symbol_emb(support[:, 1])  # (batch, embed_dim)
        support_e = self.entity_encoder(support_e1, support_e2)

        query_left = self.neighbor_encoder(query_left_connections, query_left_degrees)
        query_right = self.neighbor_encoder(query_right_connections, query_right_degrees)

        support_left = self.neighbor_encoder(support_left_connections, support_left_degrees)
        support_right = self.neighbor_encoder(support_right_connections, support_right_degrees)

        query_neighbor = torch.cat((query_left, query_e, query_right), dim=-1)  # tanh
        support_neighbor = torch.cat((support_left, support_e, support_right), dim=-1)  # tanh

        support = support_neighbor
        query = query_neighbor

        support_g = self.support_encoder(support)  # 1 * 100
        query_g = self.support_encoder(query)



        support_g = torch.mean(support_g, dim=0, keepdim=True)

        # cosine similarity
        matching_scores = torch.matmul(query_g, support_g.t()).squeeze()

        return query_g, matching_scores



class DeViSE(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, p):
        super(DeViSE, self).__init__()
        self.model = nn.Sequential(nn.BatchNorm1d(input_dims),
                         nn.Dropout(p),
                         nn.Linear(in_features=input_dims, out_features=hidden_dims, bias=True),
                         nn.ReLU(),
                         nn.BatchNorm1d(hidden_dims),
                         nn.Dropout(p),
                         nn.Linear(in_features=hidden_dims, out_features=output_dims, bias=True))
    def forward(self, x):
        x = self.model(x)
        return x

**6. Model Training and Testing**

In [13]:
class Runner:
    def __init__(self, args):
        self.args = args

        print('############## LOADING .... ###################')

        self.train_tasks = json.load(open(os.path.join(args.data_path, 'datasplit', 'train_tasks.json')))
        self.rel2id = json.load(open(os.path.join(args.data_path, 'relation2ids')))

        self.rela_matrix = load_semantic_embed(args.data_path, args.dataset, args.semantic_type)
        self.input_dims = self.rela_matrix.shape[1]

        print('##LOADING ENTITY##')
        self.ent2id = json.load(open(os.path.join(args.data_path, 'entity2id')))
        num_ents = len(self.ent2id.keys())

        print('##LOADING CANDIDATES ENTITIES##')
        self.rel2candidates = json.load(open(os.path.join(args.data_path, 'rel2candidates_all.json')))

        # load answer dict
        self.e1rel_e2 = defaultdict(list)
        self.e1rel_e2 = json.load(open(os.path.join(args.data_path, 'e1rel_e2_all.json')))


        self.rela2label = dict()
        rela_sorted = sorted(list(self.train_tasks.keys()))
        for i, rela in enumerate(rela_sorted):
            self.rela2label[rela] = int(i)

        print('##LOADING SYMBOL ID AND SYMBOL EMBEDDING')
        self.symbol2id, self.symbol2vec = self.read_embed()

        num_symbols = len(self.symbol2id.keys()) - 1  #
        print("num symbols:", num_symbols)
        pad_id = num_symbols

        # Pretraining step to obtain reasonable real data embeddings, load already pre-trained
        print('Load Pretrained Feature Encoder!')
        feature_encoder = Extractor(args.embed_dim, num_symbols, embed=self.symbol2vec).cuda()
        feature_encoder.apply(weights_init)
        model_path = os.path.join(args.data_path, 'FE_models_trained', args.embed_model + '_Extractor')
        feature_encoder.load_state_dict(torch.load(model_path, map_location='cuda:0'))
        self.feature_encoder = feature_encoder
        self.feature_encoder.eval()


        print('##BUILDING CONNECTION MATRIX')
        self.degrees, self.connections, self.e1_degrees = self.build_connection(num_ents, pad_id, self.symbol2id, self.ent2id, max_=args.max_neighbor)

        print('################ DEFINE ZSL MODEL AND LOSS FUNCTION ... ##############')
        self.zsl_model = DeViSE(self.input_dims, args.hidden_dims, args.output_dims, args.p).cuda()
        self.loss_fn = nn.MSELoss()
        self.optimizer_tag = optim.Adam(self.zsl_model.parameters(), lr=args.lr, weight_decay=args.wds)
        print('using {} as criterion'.format(self.loss_fn))







    def read_embed(self):
        symbol_id = json.load(open(os.path.join(self.args.data_path, 'Embed_used', args.embed_model + '2id')))
        embeddings = np.load(os.path.join(self.args.data_path, 'Embed_used', args.embed_model + '.npz'))['arr_0']
        symbol2id = symbol_id
        symbol2vec = embeddings
        return symbol2id, symbol2vec

    #  build neighbor connection
    def build_connection(self, num_ents, pad_id, symbol2id, ent2id, max_=100):

        connections = (np.ones((num_ents, max_, 2)) * pad_id).astype(int)
        e1_rele2 = defaultdict(list)
        e1_degrees = defaultdict(int)
        # rel_list = list()
        with open(os.path.join(self.args.data_path, 'path_graph')) as f:
            lines = f.readlines()
            for line in tqdm(lines):
                e1, rel, e2 = line.rstrip().split()
                e1_rele2[e1].append((symbol2id[rel], symbol2id[e2]))
                e1_rele2[e2].append((symbol2id[rel], symbol2id[e1]))

        # print("path graph relations:", len(set(rel_list)))
        degrees = {}
        for ent, id_ in ent2id.items():
            neighbors = e1_rele2[ent]
            if len(neighbors) > max_:
                neighbors = neighbors[:max_]
            # degrees.append(len(neighbors))
            degrees[ent] = len(neighbors)
            e1_degrees[id_] = len(neighbors)  # add one for self conn
            for idx, _ in enumerate(neighbors):
                connections[id_, idx, 0] = _[0]
                connections[id_, idx, 1] = _[1]


        return degrees, connections, e1_degrees

    def get_meta(self, left, right, connections, e1_degrees):
        left_connections = Variable(
            torch.LongTensor(np.stack([connections[_, :, :] for _ in left], axis=0))).cuda()
        left_degrees = Variable(torch.FloatTensor([e1_degrees[_] for _ in left])).cuda()

        right_connections = Variable(
            torch.LongTensor(np.stack([connections[_, :, :] for _ in right], axis=0))).cuda()
        right_degrees = Variable(torch.FloatTensor([e1_degrees[_] for _ in right])).cuda()

        return (left_connections, left_degrees, right_connections, right_degrees)



    def train(self):


        print('\n############ START TRAINING ... ############')
        losses = deque([], args.loss_every)


        train_data = load_train_data(self.args, self.train_tasks, self.symbol2id, self.ent2id,
                                           self.e1rel_e2, self.rel2id, self.rela2label, self.rela_matrix)



        for epoch in range(1, (args.train_times+1)):
            self.zsl_model.train()
            
            rel_sem, rel_sem_neg, query, query_left, query_right, false, false_left, false_right, labels = train_data.__next__()

            rel_sem = Variable(torch.FloatTensor(rel_sem)).cuda()


            # encoding (h, t) pair
            query_meta = self.get_meta(query_left, query_right, self.connections, self.e1_degrees)
            query = Variable(torch.LongTensor(query)).cuda()

            entity_pair_vector, _ = self.feature_encoder(query, query, query_meta, query_meta)

            self.zsl_model.zero_grad()

            rel_sem_mapped = self.zsl_model(rel_sem)
            loss = self.loss_fn(entity_pair_vector, rel_sem_mapped)


            loss.backward()
            self.optimizer_tag.step()


            losses.append(loss.item())



            if epoch % args.loss_every == 0:

                print("Epoch: %d, loss: %.3f" % (epoch, np.mean(losses)))


            if epoch >= 1000 and epoch % args.eval_every == 0:
                self.test(epoch)
                # self.save_model()

    def test(self, epoch=0):
        self.zsl_model.eval()

        # test_candidates = json.load(open(os.path.join(self.args.data_path, "test_candidates_sub_10.json")))
        test_candidates = json.load(open(os.path.join(self.args.data_path, "test_candidates.json")))

        hits10 = []
        hits5 = []
        hits1 = []
        mrr = []

        for query_ in sorted(test_candidates.keys()):


            hits10_ = []
            hits5_ = []
            hits1_ = []
            mrr_ = []

            rel_sem = self.rela_matrix[self.rel2id[query_]]
            rel_sem = np.expand_dims(rel_sem, axis=0)
            rel_sem = Variable(torch.FloatTensor(rel_sem)).cuda()

            rel_sem_mapped = self.zsl_model(rel_sem)
            rel_sem_mapped.detach()
            rel_sem_mapped = rel_sem_mapped.data.cpu().numpy()

            for e1_rel, tail_candidates in test_candidates[query_].items():
                if args.dataset == "NELL":
                    head, rela, _ = e1_rel.split('\t')
                elif args.dataset == "Wiki":
                    head, rela = e1_rel.split('\t')
                else:
                    print('The Dataset is not supported, please check')

                true = tail_candidates[0]
                query_pairs = []
                if head not in self.symbol2id or true not in self.symbol2id:
                    continue
                query_pairs.append([self.symbol2id[head], self.symbol2id[true]])

                query_left = []
                query_right = []
                query_left.append(self.ent2id[head])
                query_right.append(self.ent2id[true])

                for tail in tail_candidates[1:]:
                    if tail not in self.symbol2id:
                        continue
                    query_pairs.append([self.symbol2id[head], self.symbol2id[tail]])
                    query_left.append(self.ent2id[head])
                    query_right.append(self.ent2id[tail])

                query = Variable(torch.LongTensor(query_pairs)).cuda()

                query_meta = self.get_meta(query_left, query_right, self.connections, self.e1_degrees)
                candidate_vecs, _ = self.feature_encoder(query, query, query_meta, query_meta)
                candidate_vecs.detach()
                candidate_vecs = candidate_vecs.data.cpu().numpy()
                scores = cosine_similarity(candidate_vecs, rel_sem_mapped)
                scores = np.squeeze(scores, axis=1)

                assert scores.shape == (len(query_pairs),)

                sort = list(np.argsort(scores))[::-1]  # ascending -> descending
                rank = sort.index(0) + 1
                if rank <= 10:
                    hits10.append(1.0)
                    hits10_.append(1.0)
                else:
                    hits10.append(0.0)
                    hits10_.append(0.0)
                if rank <= 5:
                    hits5.append(1.0)
                    hits5_.append(1.0)
                else:
                    hits5.append(0.0)
                    hits5_.append(0.0)
                if rank <= 1:
                    hits1.append(1.0)
                    hits1_.append(1.0)
                else:
                    hits1.append(0.0)
                    hits1_.append(0.0)
                mrr.append(1.0 / rank)
                mrr_.append(1.0 / rank)



        print('\n############   ' + 'TEST' + ' ' + str(epoch) + '    #############')
        print('HITS10: {:.3f}, HITS5: {:.3f}, HITS1: {:.3f}, MAP: {:.3f}'.format(np.mean(hits10),
                                                                                 np.mean(hits5),
                                                                                 np.mean(hits1),
                                                                                 np.mean(mrr)))
        print('###################################')

In [ ]:
run = Runner(args)
run.train()

############## LOADING .... ###################
##LOADING ENTITY##
##LOADING CANDIDATES ENTITIES##
##LOADING SYMBOL ID AND SYMBOL EMBEDDING
num symbols: 65748
Load Pretrained Feature Encoder!
##BUILDING CONNECTION MATRIX


100%|██████████| 181053/181053 [00:00<00:00, 534884.32it/s]


################ DEFINE ZSL MODEL AND LOSS FUNCTION ... ##############
using MSELoss() as criterion

############ START TRAINING ... ############
##LOADING CANDIDATES
Epoch: 50, loss: 1.498
Epoch: 100, loss: 1.092
Epoch: 150, loss: 0.816
Epoch: 200, loss: 0.722
Epoch: 250, loss: 0.693
Epoch: 300, loss: 0.677
Epoch: 350, loss: 0.676
Epoch: 400, loss: 0.644
Epoch: 450, loss: 0.653
Epoch: 500, loss: 0.637
Epoch: 550, loss: 0.640
Epoch: 600, loss: 0.642
Epoch: 650, loss: 0.628
Epoch: 700, loss: 0.633
Epoch: 750, loss: 0.628
Epoch: 800, loss: 0.625
Epoch: 850, loss: 0.619
Epoch: 900, loss: 0.606
Epoch: 950, loss: 0.615
Epoch: 1000, loss: 0.618

############   TEST 1000    #############
HITS10: 0.346, HITS5: 0.285, HITS1: 0.153, MAP: 0.220
###################################
Epoch: 1050, loss: 0.616
Epoch: 1100, loss: 0.626
Epoch: 1150, loss: 0.636
Epoch: 1200, loss: 0.618
Epoch: 1250, loss: 0.608
Epoch: 1300, loss: 0.615
Epoch: 1350, loss: 0.606
Epoch: 1400, loss: 0.608
Epoch: 1450, loss: 0

**Parameters in other Settings**


---

*   **run DeViSE on NELL with "RDFS+literals"**

------HYPERPARAMETERS-------
data_dir: /content/drive/MyDrive/ISWC_demo/ZS_KGC/data;
dataset: NELL;
embed_model: TransE;
max_neighbor: 50;
embed_dim: 100;
ep_dim: 200;
batch_size: 64;
batch_rela_num: 4;
train_times: 7000;
loss_every: 50;
eval_every: 500;
p: 0.5;
lr: 0.001;
wds: 1e-05;
manual_seed: 12345;
semantic_type: rdfs_text;
hidden_dims: 400;

*   **run DeViSE on Wiki with "RDFS"**

------HYPERPARAMETERS-------
data_dir: /content/drive/MyDrive/ISWC_demo/ZS_KGC/data;
dataset: Wiki;
embed_model: TransE;
max_neighbor: 50;
embed_dim: 50;
ep_dim: 100;
batch_size: 64;
batch_rela_num: 8;
train_times: 7000;
loss_every: 50;
eval_every: 500;
p: 0.5;
lr: 0.001;
wds: 1e-05;
manual_seed: 12345;
semantic_type: rdfs;
hidden_dims: 200;

*   **run DeViSE on Wiki with "RDFS+literals"**

------HYPERPARAMETERS-------
data_dir: /content/drive/MyDrive/ISWC_demo/ZS_KGC/data;
dataset: Wiki;
embed_model: TransE;
max_neighbor: 50;
embed_dim: 50;
ep_dim: 100;
batch_size: 64;
batch_rela_num: 8;
train_times: 7000;
loss_every: 50;
eval_every: 500;
p: 0.5;
lr: 0.001;
wds: 1e-05;
manual_seed: 12345;
semantic_type: rdfs_text;
hidden_dims: 200;


---


**The best parameter of hidden_dim for NELL**

is set to 300 for rdfs, rdfs_hie, rdfs_cons and text, is set to 400 for rdfs_text

**The best parameter of hidden_dim for Wiki**

is set to 200 for rdfs, rdfs_hie, rdfs_cons, text and rdfs_text